<a href="https://colab.research.google.com/github/Javigomez4/my-second-repo/blob/main/Problem_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

import pulp

# Problem data
W = 100
widths = [40, 20, 10]
demand = [8, 7, 10]

# Initial patterns (each pattern uses only one width type fully)
patterns = [
    [2, 0, 0],   # Pattern 1: two 40" pieces
    [0, 5, 0],   # Pattern 2: five 20" pieces
    [0, 0, 10]   # Pattern 3: ten 10" pieces
]
print("Initial patterns (each pattern uses one width type):")
for pat in patterns:
    print(" ", pat)

# Solve the master LP with current patterns
master_lp = pulp.LpProblem("CutStock_MasterLP", pulp.LpMinimize)
x = [pulp.LpVariable(f"x_{j}", lowBound=0) for j in range(len(patterns))]
# Objective: minimize total rolls
master_lp += pulp.lpSum(x)
# Constraints: meet each demand
for i in range(len(widths)):
    master_lp += pulp.lpSum(patterns[j][i] * x[j] for j in range(len(patterns))) >= demand[i], f"Demand_{i}"
master_lp.solve(pulp.PULP_CBC_CMD(msg=0))

# LP solution and dual prices
obj_val = pulp.value(master_lp.objective)
duals = [master_lp.constraints[f"Demand_{i}"].pi for i in range(len(widths))]
print(f"\nInitial LP objective = {obj_val:.2f} rolls")
for j, pat in enumerate(patterns):
    if x[j].value() > 1e-6:
        print(f"  Use pattern {pat} -> {x[j].value():.2f} rolls")
print("Dual prices:", [round(d,3) for d in duals])

# Branch-and-Bound Knapsack solver to find new pattern (maximize sum(dual_i * pieces_i))
best_pattern = None
best_value = 0

def knapsack_bnb(index, remaining_cap, current_val, pattern_count, duals):
    global best_value, best_pattern
    if index == len(widths):
        # At a leaf (pattern fully defined), check if best
        if current_val > best_value + 1e-9:
            best_value = current_val
            best_pattern = pattern_count.copy()
        return
    # Maximum pieces of item 'index' that can fit in remaining capacity
    max_count = remaining_cap // widths[index]
    # Try all counts from max_count down to 0 for this item
    for count in range(max_count, -1, -1):
        new_val = current_val + duals[index] * count
        new_cap = remaining_cap - widths[index] * count
        # Upper bound: fill remaining capacity fractionally with highest dual/width items
        bound_val = new_val
        cap_temp = new_cap
        # Determine remaining items sorted by value-per-inch (dual/width)
        rem_items = list(range(index, len(widths)))
        rem_items.sort(key=lambda k: duals[k] / widths[k], reverse=True)
        for k in rem_items:
            if cap_temp <= 0:
                break
            # Take as many of item k as fits, then possibly a fraction of one
            fit = cap_temp // widths[k]
            bound_val += fit * duals[k]
            cap_temp -= fit * widths[k]
            if cap_temp > 0:
                bound_val += (cap_temp / widths[k]) * duals[k]
                cap_temp = 0
        # Prune if even the upper bound is not better than current best
        if bound_val < best_value - 1e-9:
            continue
        # Branch: fix this count and move to next item
        pattern_count[index] = count
        knapsack_bnb(index+1, new_cap, new_val, pattern_count, duals)
        pattern_count[index] = 0

# Column generation iterations
iteration = 1
while True:
    # Solve LP with current patterns
    master_lp = pulp.LpProblem("CutStock_MasterLP", pulp.LpMinimize)
    x = [pulp.LpVariable(f"x_{j}", lowBound=0) for j in range(len(patterns))]
    master_lp += pulp.lpSum(x)
    for i in range(len(widths)):
        master_lp += pulp.lpSum(patterns[j][i] * x[j] for j in range(len(patterns))) >= demand[i], f"Demand_{i}"
    master_lp.solve(pulp.PULP_CBC_CMD(msg=0))
    obj_val = pulp.value(master_lp.objective)
    duals = [master_lp.constraints[f"Demand_{i}"].pi for i in range(len(widths))]

    # Solve knapsack subproblem using BnB
    best_pattern = None
    best_value = 0
    knapsack_bnb(0, W, 0, [0]*len(widths), duals)
    reduced_cost = 1 - best_value  # cost of a roll is 1

    # Output iteration info
    print(f"Iter {iteration}: Obj = {obj_val:.4f}, Duals = {[round(d,3) for d in duals]}, Reduced cost = {reduced_cost:.4f}")
    if reduced_cost < -1e-6 and best_pattern is not None:
        patterns.append(best_pattern)  # add new pattern to master
        print(f"  Added new pattern: {best_pattern}")
    else:
        print("  No improving pattern found. Converged.")
        break
    iteration += 1


print(f"\nFinal LP Objective (approx.): {obj_val:.1f} rolls")
print("Final patterns used:")
for pat in patterns:
    print(" ", pat)

# Solve integer master problem with found patterns
master_ip = pulp.LpProblem("CutStock_MasterIP", pulp.LpMinimize)
y = [pulp.LpVariable(f"y_{j}", lowBound=0, cat="Integer") for j in range(len(patterns))]
master_ip += pulp.lpSum(y)
for i in range(len(widths)):
    master_ip += pulp.lpSum(patterns[j][i] * y[j] for j in range(len(patterns))) >= demand[i]
master_ip.solve(pulp.PULP_CBC_CMD(msg=0))

print("\nInteger Optimal Solution:")
for j, pat in enumerate(patterns):
    if y[j].value() is not None and y[j].value() > 0:
        print(f"Pattern {pat} -> {int(y[j].value())} rolls")
print(f"\nInteger optimal number of rolls: {int(pulp.value(master_ip.objective))}")


Initial patterns (each pattern uses one width type):
  [2, 0, 0]
  [0, 5, 0]
  [0, 0, 10]

Initial LP objective = 6.40 rolls
  Use pattern [2, 0, 0] -> 4.00 rolls
  Use pattern [0, 5, 0] -> 1.40 rolls
  Use pattern [0, 0, 10] -> 1.00 rolls
Dual prices: [0.5, 0.2, 0.1]
Iter 1: Obj = 6.4000, Duals = [0.5, 0.2, 0.1], Reduced cost = -0.2000
  Added new pattern: [2, 1, 0]
Iter 2: Obj = 5.6000, Duals = [0.4, 0.2, 0.1], Reduced cost = 0.0000
  No improving pattern found. Converged.

Final LP Objective (approx.): 5.6 rolls
Final patterns used:
  [2, 0, 0]
  [0, 5, 0]
  [0, 0, 10]
  [2, 1, 0]

Integer Optimal Solution:
Pattern [0, 5, 0] -> 1 rolls
Pattern [0, 0, 10] -> 1 rolls
Pattern [2, 1, 0] -> 4 rolls

Integer optimal number of rolls: 6
